### Evaluation of criticality metrics on the nuscenes dataset.

**Results produced for the nuscenes dataset are needed, as for example they can be produced by the notebook MMDetection3D, see the other notebook MMDetection3D**

**Note: nuscenes-dev must be correctly installed**

**Note: in case of failure: loaders.py: WARNING: forced equivalence box_cls == DetectionBox --> just restart the notebook kernel**


**Use conda environments:**
conda activate nuscene
conda activate mmm --> should be sufficient

#### Some hints on the modifications done to the nuscenes library:

- detection/algo.py : the method "accumulate" computes precision, recall and average precision. The approach to compute precision and recall is fascinating. NPOS=TP+FN is a static value and can be computed immediately. Then, an array with all the detection thresholds for tps is computed e.g. $[1\; 0\; 0\; 0\; 1]$. Same is done for fp. Same is done with the detection confidence. Then they are sorted according to the detection confidence, and elements are picked up from top to bottom. Precision and recall are computed with the retrieved items, iteratively. This is the equivalent of starting getting "detected items" with the highest confidence, and progressively get all the remaining items, and compute precision and recall at each step with all the items selected so far. To compute the precision curve, there is a marvellous interpolation of the precision matrix and the recall matrix with a predefined matrix.


- detection/evaluate.py : contains the main method


- detection/data_classes.py : DetectionBox, in the init method, it includes all the logic to perform computation of all criticalities for a target BoundingBox i.e., for each object. This is done independently if it is a ground truth bounding box or a predicted bounding box. So whenever we plan to change our geometry, we need to act here.


Further, image creation is manipulated so that it shows criticality values matched to each bounding box that is depicted.  It visualizes only one type of objects (CAR), for clarity of the image --> to change this, in utils/data_classes.py, render_crit, change the line "if(self.name!='car'):"

Programmatically, the base is visualize_sample_crit in detection/render.py; then there is method render_crit; and finally we build object Box in utils/data_classes.py; images are created at the beginning of main in detection/evaluate.py; the label is added in render_crit in utils/data_classes.py. To print a different class (e.g., car, bus, everything), should be easy working with the above files (the no_crit case prints everything is on map).

**Debug images** Debug images (folder examples_debug_1) are introduced. They consider only ground truth, and they print: velocity and position of all vehicles,including the ego vehicle. They are created as follows:
- in detection/evaluate.py there is method visualize_sample_debug_1 of eval.detection.render
- in eval.detection.render method visualize_sample_debug_1, for each boxes in gt, there is render_debug_1 of utils/data_classes.py
- utils/data_classes.py method render_debug_1 builds the squares of the different ground truth cars

#### Which results are computed

Results are saved in different folders. They are easy to find, you just need to configure the proper path as it will be explained a bit later here.


We collect results in terms of:

- precision recall curve, that we save as pdf files (see the pdf files with "crit" in the name)

We compute some more metrics in a text file, that is called "confusion matrix.txt". The file contains: i) average precision; ii) average precision crit;

*Example: class_name bus; dist_th 1.0; ap 0.16393238098925264; ap_crit 0.16604208258718856*

iii) MAXIMUM number of tp and fp; iv) MAXIMUM value of tp_crit pred and MAXIMUM value of tp_crit gt; iii) MAXIMUM number of tp; iv) MAXIMUM value of fp_crit pred; v) MINIMUM number of FN, and MINIMUM value of fn_crit gt

*Example: class_name car; dist_th 0.5; max tp 19758.0; max fp 177143.0; min fn 33285.0; max tp_pred_crit 19249.891210080583; max tp_gt_crit 19272.72239514313; max fp_pred_crit 172455.20687223336; min fn_gt_crit 32468.569145629514*

To enrich this with multiple metrics, just operate on method *accumulate* in *algo.py*

Last, you can save bird eye view images that have the crit values added to each represented bounding box.

*reminder:* if you ever need to change dist_ths, dist_thp: check configs/detection_cvpr_2019.json

#### Below are configuration items

In [1]:
## POINTPILLARS WITH SECFPN BACKBONE -- lidar only
PATH='/home/notebook/nuscene/tmp/mmdetection3d/pointpillars_nuscenes_results/pts_bbox/'
FILE_JSON='results_nusc-secfpn.json'
DETECTOR='pointpillars-secfpn'

## POINTPILLARS WITH FPN BACKBONE -- lidar only
##PATH='/home/notebook/nuscene/tmp/mmdetection3d/pointpillars_nuscenes_results/pts_bbox/' 
##FILE_JSON='results_nusc-fpn.json'
##DETECTOR='pointpillars-fpn'

##RegNET WITH BACKBONE RegNetX-1.6gF-FPN
#PATH='/home/notebook/nuscene/tmp/mmdetection3d/regnetX_nuscenes_results-secfpn/pts_bbox/' 
#FILE_JSON='results_nusc-secfpn.json'
#DETECTOR='regnet-regnetX_FPN'

##RegNET WITH BACKBONE RegNET WITH RegNetX-400MF-FPN
##PATH='/home/notebook/nuscene/tmp/mmdetection3d/regnetX_nuscenes_results-secfpn/pts_bbox/' 
##FILE_JSON='results_nusc-400MF.json'
##DETECTOR='regnet-regnetX_400MF-FPN'

##RegNET WITH BACKBONE RegNET WITH RegNetX-400MF-SECFPN
##PATH='/home/notebook/nuscene/tmp/mmdetection3d/regnetX_nuscenes_results-secfpn/pts_bbox/' 
##FILE_JSON='results_nusc-400mfsecfpn.json'
##DETECTOR='regnet-regnetX_400MF-SECFPN'

#### SSN con backbone SECFPN -- LIDAR-only
##PATH='/home/notebook/nuscene/tmp/mmdetection3d/ssn_nuscenes_results/pts_bbox/' 
##FILE_JSON='results_nusc-secfpn.json'
##DETECTOR='ssn-SECFPN'

#### SSN con backbone REGNET -- LIDAR-only
##PATH='/home/notebook/nuscene/tmp/mmdetection3d/ssn_nuscenes_results/pts_bbox/' 
##FILE_JSON='results_nusc-regnet.json'
##DETECTOR='ssn-REGNET'

### FCOSD con backbone resnet 101 (finetuned) -- camera only
##PATH='/home/notebook/mmdetection3d/fcos_results/img_bbox/' 
##FILE_JSON='results_nusc.json'
##DETECTOR='FCOSD-RESNET101'

#nuscene dataroot
DATAROOT = '/home/notebook/nuscene/data'
#path + json file where detection results from mmdetection3d are stored, ready to be processed
#PATH='/home/notebook/nuscene/tmp/mmdetection3d/pointpillars_nuscenes_results/pts_bbox/'
#FILE_JSON='results_nusc-secfpn.json'
#DETECTOR='pointpillars-secfpn'
RESULT_PATH=PATH+FILE_JSON
#the detector in use. Does nothing special but creates a folder, and it is useful to put results there
#results of evaluation will be stored here
OUTPUT='/home/notebook/dsn/results/'+DETECTOR+"/"

#parameters from our marvellous solution. At the bottom of the notebook, there is a different 
#approach where multiple config are tested
MAX_DISTANCE_OBJ_1=30.0
MAX_DISTANCE_INTERSECT_1=20.0
MAX_TIME_INTERSECT_OBJ_1=10.0

#how many images in bird view you want to draw
NUMBER_IMAGE=1

In [2]:
#required if you are playing with libraries and changing them. This reloads libraries
%load_ext autoreload
%autoreload 2

In [3]:
import unittest
import numpy as np
import sklearn
import tqdm
import pandas
import math
import json
from typing import Callable
from nuscenes import NuScenes
from nuscenes.eval.prediction.splits import *
import nuscenes.eval.detection.config as cnfig
from nuscenes.eval.detection.configs import *
from nuscenes.eval.detection.data_classes import DetectionBox 
from nuscenes.eval.detection import *
import nuscenes.eval.detection.algo as ag
from nuscenes.eval.detection.data_classes import DetectionMetricData, DetectionConfig, DetectionMetrics, DetectionBox, \
    DetectionMetricDataList
from nuscenes.eval.common.data_classes import EvalBoxes

import os
from typing import List, Dict, Callable, Tuple
from nuscenes.eval.common.utils import center_distance, scale_iou, yaw_diff, velocity_l2, attr_acc, cummean
import nuscenes.eval.detection.evaluate as dcl    
from nuscenes.prediction import *
from nuscenes.map_expansion.map_api import *

In [4]:
nuscenes = NuScenes('v1.0-trainval', dataroot=DATAROOT)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 28.944 seconds.
Reverse indexing ...
Done reverse indexing in 7.6 seconds.


In [5]:
confvalue=cnfig.config_factory("detection_cvpr_2019")

In [6]:
#number of scene that compose the val set
eval=val = \
    ['scene-0003', 'scene-0012', 'scene-0013', 'scene-0014', 'scene-0015', 'scene-0016', 'scene-0017', 'scene-0018',
     'scene-0035', 'scene-0036', 'scene-0038', 'scene-0039', 'scene-0092', 'scene-0093', 'scene-0094', 'scene-0095',
     'scene-0096', 'scene-0097', 'scene-0098', 'scene-0099', 'scene-0100', 'scene-0101', 'scene-0102', 'scene-0103',
     'scene-0104', 'scene-0105', 'scene-0106', 'scene-0107', 'scene-0108', 'scene-0109', 'scene-0110', 'scene-0221',
     'scene-0268', 'scene-0269', 'scene-0270', 'scene-0271', 'scene-0272', 'scene-0273', 'scene-0274', 'scene-0275',
     'scene-0276', 'scene-0277', 'scene-0278', 'scene-0329', 'scene-0330', 'scene-0331', 'scene-0332', 'scene-0344',
     'scene-0345', 'scene-0346', 'scene-0519', 'scene-0520', 'scene-0521', 'scene-0522', 'scene-0523', 'scene-0524',
     'scene-0552', 'scene-0553', 'scene-0554', 'scene-0555', 'scene-0556', 'scene-0557', 'scene-0558', 'scene-0559',
     'scene-0560', 'scene-0561', 'scene-0562', 'scene-0563', 'scene-0564', 'scene-0565', 'scene-0625', 'scene-0626',
     'scene-0627', 'scene-0629', 'scene-0630', 'scene-0632', 'scene-0633', 'scene-0634', 'scene-0635', 'scene-0636',
     'scene-0637', 'scene-0638', 'scene-0770', 'scene-0771', 'scene-0775', 'scene-0777', 'scene-0778', 'scene-0780',
     'scene-0781', 'scene-0782', 'scene-0783', 'scene-0784', 'scene-0794', 'scene-0795', 'scene-0796', 'scene-0797',
     'scene-0798', 'scene-0799', 'scene-0800', 'scene-0802', 'scene-0904', 'scene-0905', 'scene-0906', 'scene-0907',
     'scene-0908', 'scene-0909', 'scene-0910', 'scene-0911', 'scene-0912', 'scene-0913', 'scene-0914', 'scene-0915',
     'scene-0916', 'scene-0917', 'scene-0919', 'scene-0920', 'scene-0921', 'scene-0922', 'scene-0923', 'scene-0924',
     'scene-0925', 'scene-0926', 'scene-0927', 'scene-0928', 'scene-0929', 'scene-0930', 'scene-0931', 'scene-0962',
     'scene-0963', 'scene-0966', 'scene-0967', 'scene-0968', 'scene-0969', 'scene-0971', 'scene-0972', 'scene-1059',
     'scene-1060', 'scene-1061', 'scene-1062', 'scene-1063', 'scene-1064', 'scene-1065', 'scene-1066', 'scene-1067',
     'scene-1068', 'scene-1069', 'scene-1070', 'scene-1071', 'scene-1072', 'scene-1073']

In [13]:
%reload_ext autoreload
dt=dcl.DetectionEval(nuscenes,confvalue, RESULT_PATH, 'val', OUTPUT, verbose=False, MAX_DISTANCE_OBJ=MAX_DISTANCE_OBJ_1,
                     MAX_DISTANCE_INTERSECT=MAX_DISTANCE_INTERSECT_1, MAX_TIME_INTERSECT_OBJ=MAX_TIME_INTERSECT_OBJ_1)

  4%|▍         | 240/6019 [00:00<00:04, 1255.47it/s]

detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
car
nan
-0.18653331355701677
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
traffic_cone
nan
-0.2162695361018028
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
bicycle
nan
8.080977644903703
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
bicycle
nan
8.080977644903703
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
car
nan
6.65092481633017
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
pedestrian
nan
2.5376609670083803
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
motorcycle
nan
4.421449534332009
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
pedestrian
nan
4.6923300809984925
detection/data_classes.py line 440, speed is NaN, us

  8%|▊         | 458/6019 [00:00<00:08, 622.68it/s] 

detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
barrier
nan
-3.671974026470071
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
pedestrian
nan
-3.671974026470071
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
barrier
nan
-3.671974026470071
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
barrier
nan
-3.671974026470071
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
barrier
nan
-3.671974026470071
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
barrier
nan
-3.671974026470071
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
pedestrian
nan
-3.811936802642937
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
pedestrian
nan
-4.032576052531365
detection/data_classes.py line 440, speed is Na

 10%|█         | 627/6019 [00:00<00:08, 659.26it/s]

...

 87%|████████▋ | 5228/6019 [00:09<00:01, 732.20it/s]

detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
traffic_cone
nan
2.558519599531595
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
car
nan
3.3556634598773525
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
car
nan
3.3556634598773525
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
traffic_cone
nan
0.08971137296271095
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
traffic_cone
nan
0.08804059225070046


 94%|█████████▎| 5629/6019 [00:09<00:00, 1012.68it/s]

detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
car
nan
-0.13772550125531013
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
car
nan
5.14770227259661
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
car
nan
5.14770227259661
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
car
nan
4.826245656013112


detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
car
nan
5.4685555767418166e-05
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
car
nan
3.6631036951099145e-05
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
car
nan
2.05204264330477e-05
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
car
nan
-4.118948269991386
detection/data_classes.py line 440, speed is NaN, use max crit_d/crit_t -- check if OK
car
nan
-5.004918097202127


In [8]:
%reload_ext autoreload
#dt.main(plot_examples=NUMBER_IMAGE,render_curves=True)
dt.main(plot_examples=NUMBER_IMAGE,
        render_curves=True, 
        model_name=DETECTOR,
        MAX_DISTANCE_OBJ=MAX_DISTANCE_OBJ_1,
        MAX_DISTANCE_INTERSECT=MAX_DISTANCE_INTERSECT_1,
        MAX_TIME_INTERSECT=MAX_TIME_INTERSECT_OBJ_1,
        recall_type="GT AL NUMERATORE") #"GT AL NUMERATORE" o "PRED AL NUMERATORE"

STARTING EVALUATION in main (self)
Rendering sample token 5376e3a2874542d8b440faa899e52b97
Rendering sample token 5376e3a2874542d8b440faa899e52b97
Rendering sample token 5376e3a2874542d8b440faa899e52b97
Rendering sample token 5376e3a2874542d8b440faa899e52b97
Rendering sample token 5376e3a2874542d8b440faa899e52b97
Rendering sample token 5376e3a2874542d8b440faa899e52b97
STARTING EVALUATION in evaluate(self)
number values:  53043 GT of class car out of 121861 total across 6019 samples.
Total value of GT crit npos_crit:  32099.975061344834  of class car out of 121861 total across 6019 samples.
Found 150217 PRED of class car out of 710264 total across 6019 samples.
number values:  53043 GT of class car out of 121861 total across 6019 samples.
Total value of GT crit npos_crit:  32099.975061344834  of class car out of 121861 total across 6019 samples.
Found 150217 PRED of class car out of 710264 total across 6019 samples.
number values:  53043 GT of class car out of 121861 total across 6019 s

number values:  1824 GT of class bicycle out of 121861 total across 6019 samples.
Total value of GT crit npos_crit:  1354.4689709055872  of class bicycle out of 121861 total across 6019 samples.
Found 46966 PRED of class bicycle out of 710264 total across 6019 samples.
number values:  1824 GT of class bicycle out of 121861 total across 6019 samples.
Total value of GT crit npos_crit:  1354.4689709055872  of class bicycle out of 121861 total across 6019 samples.
Found 46966 PRED of class bicycle out of 710264 total across 6019 samples.
number values:  1824 GT of class bicycle out of 121861 total across 6019 samples.
Total value of GT crit npos_crit:  1354.4689709055872  of class bicycle out of 121861 total across 6019 samples.
Found 46966 PRED of class bicycle out of 710264 total across 6019 samples.
number values:  10570 GT of class traffic_cone out of 121861 total across 6019 samples.
Total value of GT crit npos_crit:  8849.330740723168  of class traffic_cone out of 121861 total across

{'label_aps': defaultdict(<function nuscenes.eval.detection.data_classes.DetectionMetrics.__init__.<locals>.<lambda>()>,
             {'car': defaultdict(float,
                          {0.5: 0.6773244586484763,
                           1.0: 0.7957209145536395,
                           2.0: 0.8334597019348283,
                           4.0: 0.8516081129133508}),
              'truck': defaultdict(float,
                          {0.5: 0.20453528302744914,
                           1.0: 0.3766159178476938,
                           2.0: 0.4735651254091999,
                           4.0: 0.5122037512627609}),
              'bus': defaultdict(float,
                          {0.5: 0.2306455277619025,
                           1.0: 0.5075581081879563,
                           2.0: 0.6538675012727101,
                           4.0: 0.6873407383600322}),
              'trailer': defaultdict(float,
                          {0.5: 0.017724324941552762,
                           1

In [9]:
just a stop condition

SyntaxError: invalid syntax (<ipython-input-9-0931e18c4adf>, line 1)

### This is automatic, to test multiple settings
It takes 1 value from each array, scanning from bottom to top, and applies the usual methods as above.
e.g. , if i=3, takes MAX_DISTANCE_OBJ_ARRAY[3], MAX_DISTANCE_INTERSECT_ARRAY[3], MAX_TIME_INTERSECT_ARRAY[3]

In [ ]:
MAX_DISTANCE_OBJ_ARRAY=      [10.0, 15.0, 5.0,  10.0, 20.0, 20.0, 30.0, 40.0, 50.0] #[50.0, 50.0, 60.0, 50.0, 50.0, 500.0, 100.0, 100.0, 1000.0, 10000.0, 5.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0, 120.0, 140.0, 150.0, 200.0, 300.0, 500.0, 800.0, 1000.0, 5000.0, 10000.0, 10000.0, 10001.0]
MAX_DISTANCE_INTERSECT_ARRAY=[10.0, 15.0, 15.0, 20.0, 20.0, 20.0, 30.0, 40.0, 50.0] #[100.0, 50.0, 100.0, 100.0, 10000.0, 50.0, 100.0, 100.0, 1000.0, 10000.0, 5.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0, 120.0, 140.0, 150.0, 200.0, 300.0, 500.0, 800.0, 1000.0, 5000.0, 10000.0, 10000.0, 10001.0]
MAX_TIME_INTERSECT_ARRAY=    [3.0, 5.0,   3.0,  5.0, 7.0,  10.0, 15.0, 20.0, 25.0] #[5.0, 5.0, 10.0, 10.0, 10000.0, 10.0, 10.0, 1000.0, 50.0, 100.0, 5.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0, 120.0, 140.0, 150.0, 200.0, 300.0, 500.0, 800.0, 1000.0, 5000.0, 10000.0, 10000.0, 10001.0]

In [ ]:
%reload_ext autoreload

for i in range(0, len(MAX_DISTANCE_OBJ_ARRAY)):
    OUTPUT1=OUTPUT+'/iteration_'+str(i)+'/'

    j=len(MAX_DISTANCE_OBJ_ARRAY)-1-i #per partire dall'ultimo, che ha il valore piu alto. Con 5, 10, al momento crasha ogni cosa

    dt=dcl.DetectionEval(nuscenes,
                         confvalue, RESULT_PATH, 'val', OUTPUT1, verbose=False,
                         MAX_DISTANCE_OBJ=MAX_DISTANCE_OBJ_ARRAY[j],
                         MAX_DISTANCE_INTERSECT=MAX_DISTANCE_INTERSECT_ARRAY[j], 
                         MAX_TIME_INTERSECT_OBJ=MAX_TIME_INTERSECT_ARRAY[j])
#    dt.main(plot_examples=NUMBER_IMAGE,render_curves=True)
    dt.main(plot_examples=NUMBER_IMAGE,render_curves=True, 
            model_name=DETECTOR,
            MAX_DISTANCE_OBJ=MAX_DISTANCE_OBJ_1,
            MAX_DISTANCE_INTERSECT=MAX_DISTANCE_INTERSECT_1,
            MAX_TIME_INTERSECT=MAX_TIME_INTERSECT_1)
    f = open(OUTPUT1+"README_CONFIG.txt", "w")
    f.write("Configurations for this iteration\n")
    f.write("MAX_DISTANCE_OBJ "+str(MAX_DISTANCE_OBJ_ARRAY[j])+"\n")
    f.write("MAX_DISTANCE_INTERSECT "+str(MAX_DISTANCE_INTERSECT_ARRAY[j])+"\n")
    f.write("MAX_TIME_INTERSECT "+str(MAX_TIME_INTERSECT_ARRAY[j])+"\n")
    f.close()
    del dt